This notebook contains the main pipeline to analyze insect images 

##### Implemented:
- Looks at the brightness of each image. If the image is to dark, it will not be analyzed and moved to the `dark_frames` folder.
- Identifies insects in the image with GroundinDino, defines a bounding-box and saves a cropped image of the insect in the `cropped` folder.
- Checks whether the detection has already appeared on one of the last images to prevent saving a cropped image of the same individuum over and over. The tracker from AMT (Automatic Moth Trap: https://stangeia.hobern.net/autonomous-moth-trap-image-pipeline/) was implemented for this task.
- AMT also includes funtionality that attempts to optimize the cropped image of an individuum by saving a new version of the insect if it is sharper than the old one.
- The code quickly checks the size of each detection. If it is unrealistically big, the crop is saved in the `potentially_faulty` folder.
- Saves the raw images with the tracking visualized on them in another subfolder called `detection_drawings` - if activated.
- Goes through all the cropped images in `cropped` again and sorts out any that are not recognized as an insect. Moves these images into the `potentially_faulty` folder. A custom trained algorithm (Apollo Environment, inference_dirt_classifier.py) is responsible for this. Room for improvement here through better training data.
- Classifies the cropped images using the InsectDetect classifier (https://github.com/maxsitt/insect-detect/tree/main?tab=readme-ov-file), ApolloNet - a self trained neural Network, or BioClip (https://imageomics.github.io/bioclip/) (https://github.com/Imageomics/pybioclip)
- Measures the length of the insects in the cropped images using a custom trained version of the SLEAP pipeline (https://sleap.ai/tutorials/initial-training.html)

In [2]:
from transformers import AutoModelForMaskGeneration, AutoProcessor, pipeline
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from PIL import Image, ImageDraw, ImageFont, PngImagePlugin
from typing import Any, List, Dict, Optional, Union, Tuple
from scipy.optimize import linear_sum_assignment
from bioclip import TreeOfLifeClassifier, Rank
from IPython.display import clear_output
import matplotlib.patches as patches
import plotly.graph_objects as go
from dataclasses import dataclass
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import deque
import ipywidgets as widgets
import plotly.express as px
import pandas as pd
import numpy as np
import subprocess
import warnings
import requests
import random
import torch
import glob
import json
import math
import time
import csv
import cv2
import os
import re

### Selfmade functions
from FunktionenZumImportieren.helper_funktionen_clean import *
from FunktionenZumImportieren.settings_widgets import *
from AMT_functions.colors import reportcolors
from AMT_functions.amt_tracker import *

warnings.simplefilter(action='always', category=UserWarning)
warnings.formatwarning = custom_warning_format
warnings.filterwarnings('ignore')

envs_path = get_local_env_path()

The envs directory is: C:\Users\rolingni\AppData\Local\anaconda3\envs


#### Define settings variables
##### Variables include:
- labels: prompt that specifies what GroundingDino searches in the images. Leave as `insect`.
- threshold: a threshold that tells GroundingDino whether or not to reject a detection or to accept it. Lower values means more detections but also more faulty detections.
- buffer: specifies the amount of border space in the cropped images. `15` is a good value usually.
- pixel_scale: how many pixels are in a cm of image on the current camera. Necessary for outputing mm readings of the insect length. Input `10` if you want to output the length in pixel.
- start_image: The name of a picture can be specified here (with extention). The code will then skip all the files before. Leave empty if you want to analyze all the images in the folder. Please leave this empty in the batch analysis pipeline. Might cause issues. If you want to start at the folder of a specific day, then put the folder number at the first spot in the brackets ([___:]) where the big arrow is in the main cell (<---). If you want to start from the 5th folder for some reason, then put in a 4.
- save_visualisation: activate if you want a visualisation of the detections and tracks saved in "detection_drawings"
- rank: Defines the taxonomic rank to which insects should be classified if BioClip is used. Can be set to None. This will make the algorithm classify up to the taxonomic rank that first satisfies the requirement set by certainty_threshold (see the function BioClip_inference in helper_funktionen). Setting rank to None will increase compute time.
- DIOPSIS_folder: diopsis camera folder containing the raw images to be analyzed. This structure is expected: DIOPSIS_folder/photos/folders_of_days

    WORD OF CAUTION: The 'score' value (in the results) of the BioClip inference will only reflect the certainty for the taxonomic rank up to which the classification went. That means that if you classified up to species level for example, and later in the analysis go up to family level, the score will no longer reflect how certain BioClip is for the family rank. The score always reflects the certainty that is achieved for the taxonomic rank that is stated in the 'highest_classification_rank' column of the results csv. If you first classified to species, but then later decide to switch your analysis to family, you need to run the inference again (ideally with the 'Only_BioClip_inference' notebook in 'utils') in order to obtain the right score values for that taxonomic rank. Otherwise, the 'score' value might appear either way to high or way to low.

In [3]:
get_values = create_interactive_widgets()

Dropdown(description='Annotation algorithm:', layout=Layout(height='30px', width='50%'), options=(('BioClip', …

Checkbox(value=True, description='Perform image classification with BioClip', indent=False)

Dropdown(description='Taxonomic rank:', index=4, layout=Layout(height='30px', width='50%'), options=(('kingdom…

Checkbox(value=False, description='Perform image classification with ApolloNet', indent=False)

Checkbox(value=False, description='Perform image classification with InsectDetect', indent=False)

Checkbox(value=False, description='Save visualisations', indent=False)

Text(value='', description='If you want to start at a specific image:', layout=Layout(height='30px', width='50…

In [3]:
labels = ["insect"]
threshold = 0.25
buffer = 15
pixel_scale = ask_for_pixel_scale()

Give the pixel scale of the current camera (in pixel per cm): 92


In [4]:
## On Mac:
DIOPSIS_folder = "/Users/rentaluser/Desktop/DIOPSIS-383"

## On Windows:
#DIOPSIS_folder = r"C:\Users\rolingni\Desktop\input_test4"

Main pipeline cell. All functionallity is included here\
Carefully monitor the printed details since some errors are not cought, but rather only printed. An error summary will appear at the end, telling you whether an error occured during the whole inference or not.

In [5]:
settings = get_values()
global_start_time = time.time()

global_error = False
classifier = TreeOfLifeClassifier()
tracker = AMTTracker(config)
object_detector = load_grounding_dino_model()

image_folders = [folder for folder in os.listdir(os.path.join(DIOPSIS_folder, "photos")) if not folder.startswith('.')]
image_folders = sorted(image_folders)

for img_folder in image_folders[:]: #  <------------------------------HERE----------------------------------------HERE-------------------------
    image_folder = os.path.join(DIOPSIS_folder, "photos", img_folder)
    start_time = time.time()
    
    start_processing = False
    blobs = []
    trails = {}
    first_pass = True
    images = os.listdir(image_folder)
    images = sorted(images)
    
    for image in tqdm(images, desc = img_folder):
        image_arg = os.path.join(image_folder, image)
        skip, start_processing = should_image_be_skipped(settings.start_image, start_processing, image_arg, image, image_folder)
        if skip:
            continue
        detections = detect(
            object_detector,
            image=image_arg,
            labels=labels,
            threshold=threshold
        )
        #print("Nr. of detections:", len(detections))
        image_array = Image.open(image_arg)
        image_array = np.array(image_array)
        blobs = convert_bounding_boxes(detections, image, image_array, image_folder, buffer)
        if first_pass:
            tracker.savedois = blobs
        new_tracks, _ = tracker.managetracks(tracker.savedois, blobs, first_pass)
        tracker.savedois = new_tracks
        first_pass = False
    
        if settings.save_visualisation:
            plot_tracks_and_detections(image_array, new_tracks, image)
    
    #print("--------")
    
    ## Classifying the cropped images
    print("Done detecting all the insects and saving cropped versions.")

    # Checking if there are any crops to be analyzed
    crops_available, crops_folder, crops_filepaths = are_crops_available(image_folder)
    
    print("Double-checking for faulty detections...")
    if crops_available:
        Apollo_script_path = os.path.join(envs_path, "ApolloNet", "Intro-to-CV-for-Ecologists-main", "inference_dirt_classifier.py")
        Apollo_command = f'conda run -n ApolloNet python "{Apollo_script_path}" "{crops_folder}"'
        Apollo = subprocess.run(Apollo_command, shell=True, capture_output=True, text=True)
        if Apollo.returncode != 0:
            print(Apollo.stderr)
            global_error = True
        else:
            matches = re.search(r"HERE (\d+)", Apollo.stdout)
            matches = int(matches.group(1))
            if matches > 0:
                print(f"Found {matches} potentially faulty crops. Moved them to the potentially_faulty folder.\n--------")
            else:
                print("All crops seem to be insects :)\n--------")
                
    crops_available, crops_folder, crops_filepaths = are_crops_available(image_folder)
    
    print("Classifying crops now.")
    
    if settings.InsectDetect and crops_available:
        print("InsectDetect classifier running...")
        os.chdir(os.path.join(envs_path, "InsectDetectSAM", "yolov5-cls"))
        print("Working directory is now:", os.getcwd())
        !python classify/predict.py --project {image_folder} --name results --source {crops_folder} --weights "insect-detect-ml-main/models/efficientnet-b0_imgsz128.onnx" --img 128 --sort-top1 --sort-prob --concat-csv
        print("--------")
    
    if settings.ApolloNet and crops_available:
        print("Performing ApolloNet Classification :)")
        Apollo_script_path = os.path.join(envs_path, "ApolloNet", "Intro-to-CV-for-Ecologists-main", "inference.py")
        Apollo_command = f'conda run -n ApolloNet python "{Apollo_script_path}" "{crops_folder}"'
        Apollo = subprocess.run(Apollo_command, shell=True, capture_output=True, text=True)
        if Apollo.returncode != 0:
            print(Apollo.stderr)
            global_error = True
        else:
            print("ApolloNet ran clean\n--------")
    
    if settings.BioClip and crops_available:
        print("The tree of life is growing: running BioClip algorithm for image classification now...")
        BioClip_predictions = BioClip_inference(classifier, crops_filepaths, settings.rank, certainty_threshold = 0.45)
        if len(BioClip_predictions) > 0:
            clean_predictions = process_BioClip_predictions(BioClip_predictions, image_folder, settings.rank)
        print("--------")
        
    if crops_available:
        print("Done classifying the insects. Measuring body lengths now...")
        length_script_path = os.path.join(envs_path, "sleap", "body_length_inference_folder.py")
        command = f'conda run -n sleap python "{length_script_path}" "{crops_folder}" "{pixel_scale}"'
        ran_clean = subprocess.run(command, shell=True, capture_output=True, text=True)
        if ran_clean.returncode != 0:
            print(f"stdout = {ran_clean.stdout}")
            traceback_index = ran_clean.stderr.find("Traceback")
            print(ran_clean.stderr[traceback_index:])
            global_error = True
        else:
            print("Length measurements ran clean\n--------")
        
        merge_result_csvs(image_folder)
        
    
    print("Done measuring. Annotating all results onto cropped images now...")
    results_csv = get_classification_results_csv(image_folder, settings.annotation_algorithm)
    input_folder = os.path.join(image_folder, "cropped_and_annotated")
    length_csv_file_path = os.path.join(image_folder, "results", "body_length_results.csv")
    
    if results_csv is not None and crops_available:
        annotate_classifications(classification_results_csv = results_csv, body_length_csv = length_csv_file_path,
                                 cropped_images_folder = input_folder, image_folder = image_folder, pixel_scale = pixel_scale,
                                annotation_algorithm = settings.annotation_algorithm)
    
    print("--------")
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"Done with day! No length Errors occured :)\nElapsed time: {elapsed_time/60:.2f} minutes \nTime per Image: {elapsed_time/len(images):.2f} seconds")

print("Merging all results csv into one...")
merge_all_results(DIOPSIS_folder)

global_end_time = time.time()
global_elapsed_time = global_end_time - global_start_time

print(f"Time elapsed in total: {(global_elapsed_time/60)/60:.2f} hours")
print(f"Pipeline took {round((global_elapsed_time/len(image_folders))/60, 2)} minutes per day on average")
if not global_error:
    print("All inferences ran clean :)")
else:
    print("WARNING: At least one inference error occured somewhere :(")

---------- GroundingDino runs on cpu ----------


20240625:   0%|          | 0/363 [00:00<?, ?it/s]

Done detecting all the insects and saving cropped versions.
Double-checking for faulty detections...
Found 5 potentially faulty crops. Moved them to the potentially_faulty folder.
--------
Classifying crops now.
The tree of life is growing: running BioClip algorithm for image classification now...
Classified all images up to family-level
--------
Done classifying the insects. Measuring body lengths now...
Length measurements ran clean
--------
Done measuring. Annotating all results onto cropped images now...
--------
Done with day! No length Errors occured :)
Elapsed time: 41.73 minutes 
Time per Image: 6.90 seconds


20240626:   0%|          | 0/639 [00:00<?, ?it/s]

Done detecting all the insects and saving cropped versions.
Double-checking for faulty detections...
Found 9 potentially faulty crops. Moved them to the potentially_faulty folder.
--------
Classifying crops now.
The tree of life is growing: running BioClip algorithm for image classification now...
Classified all images up to family-level
--------
Done classifying the insects. Measuring body lengths now...
Length measurements ran clean
--------
Done measuring. Annotating all results onto cropped images now...
--------
Done with day! No length Errors occured :)
Elapsed time: 76.27 minutes 
Time per Image: 7.16 seconds


...

Done detecting all the insects and saving cropped versions.
No crops available for further analysis
Double-checking for faulty detections...
No crops available for further analysis
Classifying crops now.
Done measuring. Annotating all results onto cropped images now...
No BioClip results found.
--------
Done with day! No length Errors occured :)
Elapsed time: 2.31 minutes 
Time per Image: 1.83 seconds


20240917:   0%|          | 0/184 [00:00<?, ?it/s]

Done detecting all the insects and saving cropped versions.
Double-checking for faulty detections...
Found 1 potentially faulty crops. Moved them to the potentially_faulty folder.
--------
Classifying crops now.
The tree of life is growing: running BioClip algorithm for image classification now...
Classified all images up to family-level
--------
Done classifying the insects. Measuring body lengths now...
Length measurements ran clean
--------
Done measuring. Annotating all results onto cropped images now...
--------
Done with day! No length Errors occured :)
Elapsed time: 11.94 minutes 
Time per Image: 3.89 seconds


20240918:   0%|          | 0/120 [00:00<?, ?it/s]

Done detecting all the insects and saving cropped versions.
Double-checking for faulty detections...
Found 2 potentially faulty crops. Moved them to the potentially_faulty folder.
--------
Classifying crops now.
The tree of life is growing: running BioClip algorithm for image classification now...
Classified all images up to family-level
--------
Done classifying the insects. Measuring body lengths now...
Length measurements ran clean
--------
Done measuring. Annotating all results onto cropped images now...
--------
Done with day! No length Errors occured :)
Elapsed time: 9.19 minutes 
Time per Image: 4.60 seconds
Merging all results csv into one...
No results for day 20240912
No results for day 20240913
No results for day 20240914
No results for day 20240915
No results for day 20240916
Time elapsed in total: 37.93 hours
Pipeline took 26.46 minutes per day on average
All inferences ran clean :)
